In [2]:
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

In [3]:
dinner_audio = pd.read_csv('./csv/dinner_audio.csv')

In [4]:
dinner_audio.head(2)

,id,name,mfcc,scem,scom,srom,sbwm,tempo,rmse
0,1,A Touch Of Silver.mp3,2.182090,2008.227306,22.166286,4271.332505,2362.906658,129.199219,4.348369
1,2,All The Joy.mp3,-0.420979,1810.502120,22.819678,3800.143688,2217.648805,89.102909,3.749902


In [5]:
dinner_track = pd.read_csv('./csv/dinner_track.csv')

In [6]:
dinner_track.head(2)

,id,name,uri,artist,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,4dn6rw5Ze1uWrLm1uOk1gu,Stockholm Sweetnin',spotify:track:4dn6rw5Ze1uWrLm1uOk1gu,"Scott Hamilton, Jesper Lundgaard, Jan Lundgren...",0.838,0.602,475680,0.302,0.907,8,0.113,-11.627,1,0.0427,119.758,4,0.300
1,6CDQBADsdzJwc3qZ3OPDHH,Almost Like Being in Love,spotify:track:6CDQBADsdzJwc3qZ3OPDHH,"Red Garland, Paul Chambers, Art Taylor,",0.873,0.571,290293,0.346,0.519,0,0.098,-12.569,0,0.0409,93.885,4,0.687


In [34]:
data_dir = './mp3'
audio_files = glob(data_dir + '/*.mp3')

In [25]:
from pydub import AudioSegment
from pydub.utils import make_chunks

myaudio = AudioSegment.from_file("./mp3/Brother Hiroshi.mp3" , "mp3") 
chunk_length_ms = 10000 # pydub calculates in millisec
chunks = make_chunks(myaudio, chunk_length_ms) #Make chunks of one sec

#Export all of the individual chunks as wav files

for i, chunk in enumerate(chunks):
    chunk_name = "chunk{0}.wav".format(i)
    print("exporting", chunk_name)
    chunk.export(chunk_name, format="wav")

exporting chunk0.wav
exporting chunk1.wav
exporting chunk2.wav
exporting chunk3.wav
exporting chunk4.wav
exporting chunk5.wav
exporting chunk6.wav
exporting chunk7.wav
exporting chunk8.wav
exporting chunk9.wav
exporting chunk10.wav
exporting chunk11.wav
exporting chunk12.wav
exporting chunk13.wav
exporting chunk14.wav
exporting chunk15.wav
exporting chunk16.wav
exporting chunk17.wav
exporting chunk18.wav
exporting chunk19.wav
exporting chunk20.wav
exporting chunk21.wav
exporting chunk22.wav
exporting chunk23.wav
exporting chunk24.wav
exporting chunk25.wav
exporting chunk26.wav
exporting chunk27.wav
exporting chunk28.wav
exporting chunk29.wav
exporting chunk30.wav
exporting chunk31.wav
exporting chunk32.wav
exporting chunk33.wav
exporting chunk34.wav
exporting chunk35.wav
exporting chunk36.wav
exporting chunk37.wav


In [35]:
len(audio_files)

38

In [36]:
data = pd.DataFrame(columns = ['name','mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse'])

In [39]:
lista = []
for i in range(len(audio_files)):
    lista.append(os.listdir('./mp3/')[i])
data.name = lista

In [40]:
lista2 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    mfcc_aux = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc = np.ndarray.mean(mfcc_aux)
    lista2.append(mfcc)
data.mfcc = lista2

In [41]:
lista3 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    scem = np.ndarray.mean(cent)
    lista3.append(scem)
data.scem = lista3

In [42]:
lista4 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    S = np.abs(librosa.stft(y))
    contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
    scom = np.ndarray.mean(contrast)
    lista4.append(scom)
data.scom = lista4

In [43]:
lista5 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    srom = np.ndarray.mean(rolloff)
    lista5.append(srom)
data.srom = lista5

In [44]:
lista6 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    sbwm = np.ndarray.mean(spec_bw)
    lista6.append(sbwm)
data.sbwm = lista6

In [45]:
lista7 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    hop_length = 512
    oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
    tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

    # Compute global onset autocorrelation
    ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
    ac_global = librosa.util.normalize(ac_global)

    # Estimate the global tempo for display purposes
    tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr, hop_length=hop_length)[0]
    lista7.append(tempo)
data.tempo = lista7

In [46]:
lista8 = []
for i in range(len(audio_files)):
    y, sr = librosa.load(audio_files[i])
    rmse_aux = librosa.feature.rms(y=y)
    rmse = np.ndarray.mean(rmse_aux)
    lista8.append(rmse)
data.rmse = lista8

In [47]:
data.shape

(38, 8)

In [48]:
data.describe()

,mfcc,scem,scom,srom,sbwm,tempo,rmse
count,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000,38.000000
mean,-0.554508,1930.384174,20.931559,4443.407505,2391.993930,125.961143,0.127900
std,3.026546,264.636618,0.837278,497.528084,97.082850,3.113144,0.024879
min,-13.722935,1638.190786,18.167202,3750.799207,2220.485178,123.046875,0.031622
25%,-0.484398,1816.677884,20.590527,4162.384401,2329.081206,123.046875,0.121290
50%,0.096024,1893.865696,20.831721,4397.719579,2386.409963,123.046875,0.131991
75%,0.588827,1961.259298,21.425492,4569.741644,2426.073573,129.199219,0.141635
max,1.485935,3326.258386,22.438009,6630.253102,2709.599524,129.199219,0.176376


In [126]:
dinner_aux = dinner_audio.loc[[10, 16, 38]]
dinner_aux

,id,name,mfcc,scem,scom,srom,sbwm,tempo,rmse
10,11,Brother Hiroshi.mp3,0.941666,1863.693864,21.795506,4347.507045,2356.991393,123.046875,4.050742
16,17,Ham Hocks and Cabbage.mp3,-6.427932,1292.149973,21.808515,2683.125442,1827.347560,123.046875,1.218587
38,39,Stockholm Sweetnin'.mp3,2.181471,1736.838316,24.179880,3616.669797,2032.189879,117.453835,5.439447


# ANOTACIONES

In [ ]:
y, sr = librosa.load(audio_files[0])

MFCC

In [30]:
mfcc_aux = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
mfcc = np.ndarray.mean(mfcc_aux)
mfcc

-6.669750396974487

In [ ]:
data

Spectral centroid

In [31]:
cent = librosa.feature.spectral_centroid(y=y, sr=sr)
scem = np.ndarray.mean(cent)
scem

1518.7160477443933

Spectral contrast

In [32]:
S = np.abs(librosa.stft(y))
contrast = librosa.feature.spectral_contrast(S=S, sr=sr)
scom = np.ndarray.mean(contrast)
scom

20.435196751306847

Spectral Roll-off

In [33]:
rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
srom = np.ndarray.mean(rolloff)
srom

3120.2717310592216

Spectral Bandwidth

In [34]:
spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
sbwm = np.ndarray.mean(spec_bw)
sbwm

1983.1518209886758

Tempo 

In [39]:
hop_length = 512
oenv = librosa.onset.onset_strength(y=y, sr=sr, hop_length=hop_length)
tempogram = librosa.feature.tempogram(onset_envelope=oenv, sr=sr, hop_length=hop_length)

# Compute global onset autocorrelation
ac_global = librosa.autocorrelate(oenv, max_size=tempogram.shape[0])
ac_global = librosa.util.normalize(ac_global)

# Estimate the global tempo for display purposes
tempo = librosa.beat.tempo(onset_envelope=oenv, sr=sr, hop_length=hop_length)[0]
tempo

123.046875

RMSE

In [44]:
rmse_aux = librosa.feature.rms(y=y)
rmse = np.ndarray.mean(rmse_aux)
rmse

0.048735216

In [41]:
S, phase = librosa.magphase(librosa.stft(y))
rms = librosa.feature.rms(S=S)
rms

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [42]:
S, phase = librosa.magphase(librosa.stft(y))
rms = librosa.feature.rms(S=S)
rms

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

,id,name,mfcc,scem,scom,srom,sbwm,tempo,rmse
10,11,Brother Hiroshi.mp3,0.941666,1863.693864,21.795506,4347.507045,2356.991393,123.046875,4.050742
16,17,Ham Hocks and Cabbage.mp3,-6.427932,1292.149973,21.808515,2683.125442,1827.347560,123.046875,1.218587
38,39,Stockholm Sweetnin'.mp3,2.181471,1736.838316,24.179880,3616.669797,2032.189879,117.453835,5.439447


In [ ]:
plt.figure(figsize=(10, 4))
librosa.display.specshow(mfcc, x_axis='time')
plt.colorbar()
plt.title('MFCC')
plt.tight_layout()